<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Projects/blob/main/EUR_USD_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
pip install yfinance

# 2. Load Data

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, GRU
from numpy import array, hstack
import matplotlib.pyplot as plt

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
df = yf.download('EURUSD=X',start=datetime.datetime(2022 , 1 , 1), end=datetime.datetime.today())
df.head()

<div style="direction:rtl">
<font color='green' size="5px">
آماده سازی دیتا
    </font>
</div>

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# define input sequence
in_seq1 = array(df['Open'])
in_seq2 = array(df['High'])
in_seq3 = array(df['Low'])
in_seq4 = array(df['Close'])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))


# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4))
# choose a number of time steps
n_steps = 7
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# 3. Build Deep Learning Model

In [ ]:
# define model
model = Sequential()
model.add(GRU(50,return_sequences=True,input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))

model.add(GRU(50,return_sequences=True,input_shape=(n_steps, n_features)))
model.add(Dropout(0.4))

model.add(LSTM(50,input_shape=(n_steps, n_features)))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=400, verbose=0)

# 4. Plot Performance

In [ ]:
# define input sequence
in_seq1 = array(df['Open'])
in_seq2 = array(df['High'])
in_seq3 = array(df['Low'])
in_seq4 = array(df['Close'])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4))

n_features = X.shape[2]
n_steps = 7
myPredict = list()

X, y = split_sequences(dataset, n_steps)

for i in range(len(X)):
    x_input = X[i]
    x_input = x_input.reshape((1, n_steps, n_features))
    myPredict.append(model.predict(x_input, verbose=0))

Open = []
for i in range(len(myPredict)):
    Open.append(myPredict[i][0][0])

pred_df = pd.DataFrame({'Open': Open})

In [ ]:
real_df = df['Open'].tail(len(pred_df)).reset_index()
real_df = real_df.drop(columns=['Date'])

plt.figure(figsize=(36,10))
plt.plot(pred_df,color='blue',label='Open Prediction')
plt.plot(real_df,color='red',label='Open Actual')
plt.legend()
plt.show;